In [1]:
# GPU: 32*40 in 14.1s = 93.4/s
# CPU: 32*8 in 19.8s = 13.0/s

In [2]:
import os
os.environ['KERAS_BACKEND'] = "tensorflow"
import sys
import numpy as np
import keras as K
import tensorflow as tf
from keras.applications.resnet50 import ResNet50
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("Keras: ", K.__version__)
print("Tensorflow: ", tf.__version__)
print("Keras using {}".format(K.backend.backend()))
print("Keras channel ordering is {}".format(K.backend.image_data_format()))

Using TensorFlow backend.


OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.13.3
Keras:  2.1.1
Tensorflow:  1.5.0-dev20171120
Keras using tensorflow
Keras channel ordering is channels_last


In [3]:
!cat /proc/cpuinfo | grep processor | wc -l

12


In [4]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla K80
Tesla K80


In [5]:
# Set channels last (default)
K.backend.set_image_data_format('channels_last')
BATCH_SIZE = 32
RESNET_FEATURES = 2048
BATCHES_GPU = 40
BATCHES_CPU = 8

In [6]:
def give_fake_data(batches):
    """ Create an array of fake data to run inference on"""
    np.random.seed(0)
    dta = np.random.rand(BATCH_SIZE*batches, 224, 224, 3).astype(np.float32)
    return dta, np.swapaxes(dta, 1, 3)

In [7]:
def yield_mb(X, batchsize):
    """ Function yield (complete) mini_batches of data"""
    for i in range(len(X)//batchsize):
        yield i, X[i*batchsize:(i+1)*batchsize]

In [8]:
# Create batches of fake data
fake_input_data_cl, fake_input_data_cf = give_fake_data(BATCHES_GPU)
print(fake_input_data_cl.shape, fake_input_data_cf.shape)

(1280, 224, 224, 3) (1280, 3, 224, 224)


In [9]:
def predict_fn(classifier, data, batchsize):
    """ Return features from classifier """
    out = np.zeros((len(data), RESNET_FEATURES), np.float32)
    for idx, dta in yield_mb(data, batchsize):
        out[idx*batchsize:(idx+1)*batchsize] = classifier.predict_on_batch(dta).squeeze()
    return out

## 1. GPU

In [ ]:
# Download Resnet weights
model = ResNet50(include_top=False)

In [ ]:
model.summary()

In [ ]:
%%time
# GPU: 14.1s
features = predict_fn(model, fake_input_data_cl, BATCH_SIZE)

## 2. CPU

In [ ]:
# Need to restart notebook

In [10]:
# Kill all GPUs ...
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [11]:
model = ResNet50(include_top=False)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [12]:
# Create batches of fake data
fake_input_data_cl, fake_input_data_cf = give_fake_data(BATCHES_CPU)
print(fake_input_data_cl.shape, fake_input_data_cf.shape)

(256, 224, 224, 3) (256, 3, 224, 224)


In [13]:
%%time
# CPU: 19.8s
features = predict_fn(model, fake_input_data_cl, BATCH_SIZE)

CPU times: user 3min 20s, sys: 14.5 s, total: 3min 35s
Wall time: 19.8 s
